In [1]:
from groq import Groq
import os
from dotenv import load_dotenv
import requests

load_dotenv()
client=Groq(api_key=os.environ['GROQ'])

ModuleNotFoundError: No module named 'llama_index'

In [2]:
messages = [
    ChatMessage(
        role="system", content:"""
        You are a medical entity recognition system. Your task is to extract information and return a JSON object with exactly three keys: "amount", "expenditure", and "status". 

1. Extract the monetary value and assign it to the "amount" key.
2. Identify the category of expenditure and assign it to the "expenditure" key. The expenditure should be classified into one of the following categories: "tests", "doctor", or "medicine".
3. If the input is related to medical expenses, set the "status" key to True. If the input is unrelated or outside your scope, set the "status" key to False.

Ensure the output is a simple JSON object with no additional text or nested structures.

"""
    ),
    ChatMessage(role="user", content="spent 1k rs on neuro surgeon"),
]
resp = llm.chat(messages)

data = resp.model_dump()  # This gives you the full response as a dictionary
message_content = data['message']['content']  # Extract the 'content' part

NameError: name 'ChatMessage' is not defined

In [31]:
chat_completion = client.chat.completions.create(
    messages=[
         {"role":"system", "content":"""
        You are a medical entity recognition system. Your task is to extract information and return a JSON object with exactly three keys: "amount", "expenditure", and "status". 

        1. Extract the monetary value and assign it to the "amount" key and value should be whole number.
        2. Identify the category of expenditure and assign it to the "expenditure" key. The expenditure should be classified into one of the following categories: "tests", "doctor", or "medicine".
        3. If the input is related to medical expenses, set the "status" key to "true". If the input is unrelated or outside your scope, set the "status" key to "false".
        
        Ensure the output is a simple JSON object with no additional text or nested structures.
        give me the json with no preamble and the value of status key will be in double quotes cumpolsurily
        """
         },
         {
            "role": "user",
            "content": "spent 13k rs on neuro surgeon",
        }
    ],
    model="llama3-70b-8192",
)

resp=chat_completion.choices[0].message.content

In [32]:
eval(resp)

{'amount': 13000, 'expenditure': 'doctor', 'status': 'true'}

In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine


db_username = 'myuser'
db_password = 'root'
db_host = 'localhost'
db_port = '5432'
db_name = 'medwell_db'

connection_string = f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

In [2]:
import pandas as pd

In [18]:
df=pd.read_sql_query('''select expense_type,amount,date(date) from patient_expense order by date desc limit 10;''',engine)

In [19]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%Y-%m-%d')

In [20]:
df.to_dict("records")

[{'expense_type': 'reports', 'amount': '500', 'date': '2024-10-26'},
 {'expense_type': 'doctor', 'amount': '10000', 'date': '2024-10-26'},
 {'expense_type': 'doctor', 'amount': '10000', 'date': '2024-10-26'}]

In [15]:
df

,expense_type,amount,date
0,reports,500,2024-10-26
1,doctor,10000,2024-10-26
2,doctor,10000,2024-10-26


In [21]:
df=pd.read_sql_query('''
SELECT expense_type,SUM(CAST(amount AS DECIMAL)) as total
from patient_expense
where user_id=43
group by expense_type
;
''',engine)

In [28]:
df.to_dict("list")

{'expense_type': ['doctor', 'reports'], 'total': [20000.0, 500.0]}

In [29]:
df=pd.read_sql_query('''
SELECT CAST(amount AS DECIMAL) as expenses
from patient_expense
where user_id=43

;
''',engine)

In [31]:
df.to_dict("list")

{'expenses': [500.0, 10000.0, 10000.0]}

In [36]:
df=pd.read_sql_query('''
SELECT to_char(date, 'Month') AS month_name,sum(CAST(amount AS DECIMAL)) as expenses,date_part('month', date) AS month
from patient_expense
where user_id=43
group by to_char(date, 'Month'),date_part('month', date)
order by date_part('month', date)
;
''',engine)[["month_name","expenses"]]

In [37]:
df.to_dict("list")

{'month_name': ['October  '], 'expenses': [20500.0]}

In [38]:
df=pd.read_sql_query('''
SELECT sum(CAST(amount AS DECIMAL)) as expenses,date_part('year', date) AS year
from patient_expense
where user_id=43
group by date_part('year', date)
order by date_part('year', date)
;
''',engine)
df.to_dict("list")

{'expenses': [20500.0], 'year': [2024.0]}

In [39]:
df=pd.read_sql_query('''
SELECT date_part('month', date) AS month,expense_type,sum(CAST(amount AS DECIMAL)) as expenses
from patient_expense
where user_id=43
group by date_part('month', date),expense_type
order by date_part('month', date)
;
;
''',engine)
df.to_dict("list")

{'month': [10.0, 10.0],
 'expense_type': ['doctor', 'reports'],
 'expenses': [20000.0, 500.0]}

In [48]:
df.to_dict("list")

{'month': [10.0, 10.0],
 'expense_type': ['doctor', 'reports'],
 'expenses': [20000.0, 500.0]}

In [61]:
grouped_df = df.groupby(by="month")

# Convert to the desired JSON-like structure
result = []

In [62]:
for month,df in grouped_df:
    print(month)
    print(df)
    month_data = {
        "month": month,
        "data": df[["expense_type","expenses"]].to_dict("list")
    }
    result.append(month_data)

10.0
   month expense_type  expenses
0   10.0       doctor   20000.0
1   10.0      reports     500.0


In [63]:
result

[{'month': 10.0,
  'data': {'expense_type': ['doctor', 'reports'],
   'expenses': [20000.0, 500.0]}}]

In [ ]:
SELECT date_part('year', date) AS month,expense_type,sum(CAST(amount AS DECIMAL)) as expenses
from patient_expense
where user_id=43
group by date_part('year', date),expense_type
order by date_part('year', date)
;